In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
import joblib
from fastapi import FastAPI
from pydantic import BaseModel
import uvicorn

# Load dataset
data = pd.read_csv('creditcard.csv')

# Features and target
X = data.drop('Class', axis=1)
y = data['Class']

# Data scaling
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Handle class imbalance
sm = SMOTE(random_state=42)
X_res, y_res = sm.fit_resample(X_scaled, y)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_res, y_res, test_size=0.3, random_state=42)

# Model training
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Predictions
y_pred = model.predict(X_test)
y_prob = model.predict_proba(X_test)[:,1]

# Evaluation
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))
print("ROC-AUC Score:", roc_auc_score(y_test, y_prob))

# Save model and scaler
joblib.dump(model, 'fraud_detection_model.pkl')
joblib.dump(scaler, 'scaler.pkl')

# API using FastAPI
app = FastAPI()

model = joblib.load('fraud_detection_model.pkl')
scaler = joblib.load('scaler.pkl')

class Transaction(BaseModel):
    features: list

@app.post('/predict')
def predict_fraud(transaction: Transaction):
    data = np.array(transaction.features).reshape(1, -1)
    data_scaled = scaler.transform(data)
    prediction = model.predict(data_scaled)
    probability = model.predict_proba(data_scaled)[0][1]
    return {"fraud_prediction": int(prediction[0]), "fraud_probability": probability}

if __name__ == "__main__":
    uvicorn.run(app, host="0.0.0.0", port=8000)

